In [2]:
import numpy as np
from geo_modules import *
from wait_time import *

/Users/HongSukhyun/.local/share/virtualenvs/getyourcoffee-cLzqEqHk/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2025-06-05 15:04:52.659 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ModuleNotFoundError: No module named 'gurobipy'

In [5]:
model_config = {
    "time_baseline": {
        "morning": 3.0,
        "lunch": 10.0,
        "afternoon": 2.0,
        "evening": 5.0,
        "night": 1.0,
    },
    "category_weights": {
        "university": 1.4,
        "residential": 0.5,
        "office": 1.3,
        "mall": 1.2,
        "default": 1.0,
    },
}


# 1. S-curve
def s_curve(sf, a=1.0, b=20.0, c=2.0, d=1.5):
    return a + (b - a) * (sf**c) / (d + sf**c)


# 2. Score Factor
def compute_score_factor(info, hour, config):
    category = info.get("category", "default")
    category_weight = config["category_weights"].get(category, 1.0)

    time_slot = get_time_slot(hour)
    time_baseline = config["time_baseline"].get(time_slot, 5.0)

    rating = max(info.get("rating", 4.0), 1.0)
    review_count = max(info.get("review_count", 5), 1)
    density = info.get("density", 0.5)

    log_reviews = np.log1p(review_count)

    sf = (
        1.0 * category_weight
        + 1.0 * (time_baseline / 10.0)
        + 1.2 * density
        + 0.4 * (log_reviews / 3.0)
        + 0.8 * ((5.0 - rating) / 4.0)
    )
    return sf, rating, density


# 3. Wait Time
def estimate_wait_time(lat, lon, hour=None, config=model_config):
    info = fetch_foursquare_info(lat, lon)
    hour = hour or int(np.datetime64("now", "h").astype(int) % 24)

    sf, rating, density = compute_score_factor(info, hour, config)
    wait_time = round(s_curve(sf), 2)

    return wait_time, rating, density

In [6]:
# Simulate data for coffe shops
from geo_modules import find_nearby_cafes

# 중심 좌표들
areas = {
    "UCLA": (34.0689, -118.4452),
    "Veteran Ave": (34.0588, -118.4473),
}

# 반경(m)과 결과 제한 수
radius = 1000
limit = 20

# 결과 저장용
cafes_by_area = {}

# 각 지역별 카페 추출
for area_name, (lat, lon) in areas.items():
    print(f"\n📍 Finding cafes near {area_name}...")
    cafes = find_nearby_cafes(lat, lon, radius_m=radius, limit=limit)
    cafes_by_area[area_name] = cafes

    for cafe in cafes:
        print(f"☕ {cafe['name']} — ({cafe['lat']:.5f}, {cafe['lon']:.5f})")

2025-06-05 15:08:26.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:26.640 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:26.640 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:26.640 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.



📍 Finding cafes near UCLA...


2025-06-05 15:08:27.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:27.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:27.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:27.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:27.862 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:27.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


☕ Bruin Buzz — (34.07019, -118.44397)
☕ Cafe Synapse — (34.06718, -118.44466)
☕ Seasnet Cafe — (34.06923, -118.44302)
☕ Lollicup — (34.07051, -118.44409)
☕ Kerckhoff Coffee House — (34.07057, -118.44338)
☕ Southern Lights — (34.06839, -118.44235)
☕ Evelyn & Mo Ostin Music Cafe — (34.07028, -118.44055)
☕ North Campus Shop — (34.07391, -118.44374)
☕ Bruin Cafe — (34.07256, -118.45031)
☕ Northern Lights — (34.07443, -118.44235)
☕ Bluestone Lane — (34.06272, -118.44497)
☕ Starbucks — (34.06271, -118.44391)
☕ Starbucks — (34.06249, -118.44739)
☕ Corner Bakery — (34.06220, -118.44559)
☕ Cafe 451 — (34.07485, -118.44128)
☕ Jimmy's Coffee House — (34.07371, -118.43943)
☕ Lu Valle Eateries — (34.07356, -118.43923)
☕ Sharetea — (34.06146, -118.44645)
☕ Untitled Café — (34.07596, -118.44094)
☕ Elysee — (34.06027, -118.44674)

📍 Finding cafes near Veteran Ave...


2025-06-05 15:08:32.960 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-05 15:08:32.961 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


☕ Lazy Daisy — (34.05998, -118.44658)
☕ Le Pain Quotidien — (34.05956, -118.44570)
☕ Elysee — (34.06027, -118.44674)
☕ Starbucks — (34.05941, -118.44475)
☕ Trimana — (34.05831, -118.44458)
☕ Sharetea — (34.06146, -118.44645)
☕ Comoncy — (34.05823, -118.44397)
☕ Profeta — (34.06032, -118.44417)
☕ Junbi — (34.06235, -118.44763)
☕ Corner Bakery — (34.06220, -118.44559)
☕ Starbucks — (34.06249, -118.44739)
☕ Ministry of Coffee — (34.06188, -118.44404)
☕ Just Boba Tea House — (34.06292, -118.44815)
☕ Espressoteric — (34.05668, -118.44281)
☕ Bluestone Lane — (34.06272, -118.44497)
☕ Starbucks — (34.06271, -118.44391)
☕ Starbucks — (34.06348, -118.44398)
☕ Starbucks — (34.06591, -118.44332)
☕ Cafe Med — (34.06591, -118.44330)
☕ Cafe Synapse — (34.06718, -118.44466)
